# Home task: pandas 

### Question 1

- Load the energy data from the file [Energy Indicators.xls](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls).
It is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013.


- It should be put into a DataFrame with the variable name of "energy"


- Make sure to exclude the footer and header information from the datafile.


- The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:<br>
`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`


- Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).


- For all countries which have missing data (e.g. data with `...`) make sure this is reflected as `np.NaN` values.


- Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`


- There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.


- Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). 
It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"


- Make sure to skip the header, and rename the following list of countries:
    - `Korea, Rep.`: `South Korea`,
    - `Iran, Islamic Rep.`: `Iran`,
    - `Hong Kong SAR, China`: `Hong Kong`


- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"


- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

Function "answer_one" should return the resulted DataFrame (20 columns and 15 entries)

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)


#### Importing modules

In [668]:
import pandas as pd
import numpy as np
import re
from IPython.display import display

#### Reading the energy data

In [669]:
energy = pd.read_excel('./datasets/energy_indicators.xls', skiprows=17, skipfooter=38)
energy.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Petajoules,Gigajoules,%
0,NaN,Afghanistan,Afghanistan,321,10,78.669280
1,NaN,Albania,Albania,102,35,100.000000
2,NaN,Algeria,Algeria,1959,51,0.551010
3,NaN,American Samoa,American Samoa,...,...,0.641026
4,NaN,Andorra,Andorra,9,121,88.695650
5,NaN,Angola,Angola,642,27,70.909090
6,NaN,Anguilla,Anguilla,2,136,0.000000
7,NaN,Antigua and Barbuda,Antigua and Barbuda,8,84,0.000000
8,NaN,Argentina,Argentina,3378,79,24.064520
9,NaN,Armenia,Armenia,143,48,28.236060


#### Inspecting the data
   1.  Checking the number of rows and columns
   2.  Getting a summary of the dataframe
   3.  Checking the data for null values
   4.  Checking the duplicates in the dataframe

In [670]:
energy.shape

(227, 6)

In [671]:
energy.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Petajoules', 'Gigajoules',
       '%'],
      dtype='object')

In [672]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  0 non-null      float64
 1   Unnamed: 1  227 non-null    object 
 2   Unnamed: 2  227 non-null    object 
 3   Petajoules  227 non-null    object 
 4   Gigajoules  227 non-null    object 
 5   %           227 non-null    float64
dtypes: float64(2), object(4)
memory usage: 10.8+ KB


In [673]:
energy.isnull().sum()

Unnamed: 0    227
Unnamed: 1      0
Unnamed: 2      0
Petajoules      0
Gigajoules      0
%               0
dtype: int64

In [674]:
energy.duplicated().sum()

0

#### Observation
1. The column names lack clarity and would benefit from more specific details.
2. The `Unnamed: 0` column appears to be empty and can be safely removed.
3. It seems that columns `Unnamed: 1` and `Unnamed: 2` likely contain redundant information, so one of them can be deleted.
4. The data types for the `Petajoules` and `Gigajoules` columns should be changed to `int64`. Also columns should be converted to a uniform unit of measurement - Gigajoules.
5. There are no duplicate entries in the dataframe.
6. While there are no null values in the dataframe, it's worth noting that the `Petajoules` and `Gigajoules` columns contain some dirty data, such as the presence of `...` symbols.


#### Data cleaning and manipulation

In [675]:
energy[energy['Unnamed: 1'] != energy['Unnamed: 2']]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Petajoules,Gigajoules,%
11,NaN,Australia,Australia1,5386,231,11.810810
42,NaN,China,China2,127191,93,19.754910
43,NaN,"China, Hong Kong Special Administrative Region","China, Hong Kong Special Administrative Region3",585,82,0.000000
44,NaN,"China, Macao Special Administrative Region","China, Macao Special Administrative Region4",47,83,0.000000
58,NaN,Denmark,Denmark5,725,129,33.537660
73,NaN,France,France6,10597,166,17.020280
83,NaN,Greenland,Greenland7,9,166,78.125000
97,NaN,Indonesia,Indonesia8,8754,35,12.189040
103,NaN,Italy,Italy9,6530,109,33.667230
105,NaN,Japan,Japan10,18984,149,10.232820


It's evident that both `Unnamed: 1` and `Unnamed: 2` contain identical country data. `Unnamed: 1` appears to be the clearer option and should be retained.

In [676]:
energy = energy.drop(energy.columns[[0, 2]], axis=1)

Let's assign names to the columns:

* `Unnamed: 1` will be labeled as `country` - the country name
* `Petajoules` as `energy_supply` - the total amount of provided energy to meet the country's needs
* `Gigajoules` as `energy_supply_per_capita` - the average energy available or consumed per individual
* `%` as `renewable_%` - the percentage of renewable energy in the overall energy supply

In [677]:
energy.columns = ['country', 'energy_supply', 'energy_supply_per_capita', 'renewable_%']

Let's change data type of `energy_supply` and `energy_supply_per_capita` and convert `energy_supply` to Gigajoules.

In [678]:
num_columns = ['energy_supply', 'energy_supply_per_capita', 'renewable_%']
energy[num_columns] = energy[energy[num_columns].map(np.isreal)][num_columns]
energy[num_columns] = energy[num_columns].astype('float64') #converting to float as columns contain NaN
energy.energy_supply = energy.energy_supply * 1000000

We should rename some countries for further work.

In [679]:
renamed_countries = {
    'Republic of Korea': 'South Korea',
    'United States of America': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'China, Hong Kong Special Administrative Region': 'Hong Kong',
}
energy.country = energy.country.replace(renamed_countries)

Checking if there are numbers and parenthesis in the `country` column.

In [680]:
def has_numbers(input_string):
    return any(map(str.isdigit, input_string))

energy.country.apply(has_numbers).sum()

0

In [681]:
energy[energy.country.str.contains('\(')]

,country,energy_supply,energy_supply_per_capita,renewable_%
24,Bolivia (Plurinational State of),3.360000e+08,32.0,31.477120
70,Falkland Islands (Malvinas),1.000000e+06,258.0,33.333330
98,Iran (Islamic Republic of),9.172000e+09,119.0,5.707721
133,Micronesia (Federated States of),2.000000e+06,21.0,2.941176
183,Sint Maarten (Dutch part),1.200000e+07,327.0,0.000000
221,Venezuela (Bolivarian Republic of),2.871000e+09,95.0,67.834520


Removing the parenthesis area in country names.

In [682]:
def remove_parenthesis_area_in_str(input_string):
    return re.sub(r" ?\([^)]+\)", "", input_string)

energy.country = energy.country.apply(remove_parenthesis_area_in_str)

#### Reading the gdp data

In [683]:
gdp = pd.read_csv('./datasets/world_bank.csv', skiprows=4)
gdp.head(10)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.276188e+09,3.395794e+09,2.610039e+09,3.126019e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.112502e+10,2.161623e+10,2.350628e+10,2.804836e+10,2.592067e+10,2.947210e+10,...,1.006526e+12,9.273485e+11,8.851764e+11,1.021043e+12,1.007196e+12,1.000834e+12,9.275933e+11,1.081998e+12,1.169484e+12,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,2.055058e+10,1.999814e+10,1.801955e+10,1.889635e+10,1.841886e+10,1.890450e+10,2.014345e+10,1.458314e+10,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.044764e+10,1.117321e+10,1.199053e+10,1.272769e+10,1.389811e+10,1.492979e+10,...,8.943225e+11,7.686447e+11,6.913634e+11,6.848988e+11,7.670257e+11,8.225384e+11,7.864600e+11,8.444597e+11,8.778633e+11,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.372444e+11,8.721930e+10,4.984049e+10,6.897277e+10,7.779294e+10,6.930911e+10,5.024137e+10,6.568544e+10,1.067136e+11,NaN
5,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.322815e+10,1.138685e+10,1.186120e+10,1.301973e+10,1.515642e+10,1.540183e+10,1.516273e+10,1.793057e+10,1.888210e+10,NaN
6,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.271686e+09,2.789881e+09,2.896610e+09,3.000162e+09,3.218420e+09,3.155149e+09,2.891001e+09,3.325145e+09,3.352033e+09,NaN
7,Arab World,ARB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.905733e+12,2.554156e+12,2.537537e+12,2.637112e+12,2.843476e+12,2.868891e+12,2.533377e+12,2.928456e+12,3.557557e+12,NaN
8,United Arab Emirates,ARE,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.141054e+11,3.702755e+11,3.692553e+11,3.905168e+11,4.270494e+11,4.179897e+11,3.494730e+11,4.150216e+11,5.075349e+11,NaN
9,Argentina,ARG,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,2.445060e+10,1.827212e+10,2.560525e+10,2.834471e+10,...,5.263197e+11,5.947493e+11,5.575323e+11,6.436284e+11,5.248199e+11,4.477547e+11,3.855404e+11,4.872271e+11,6.327703e+11,NaN


#### Inspecting the data

In [684]:
gdp.shape

(266, 68)

In [685]:
gdp.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       'Unnamed: 67'],
      dtype='object')

In [686]:
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 68 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1960            132 non-null    float64
 5   1961            134 non-null    float64
 6   1962            136 non-null    float64
 7   1963            136 non-null    float64
 8   1964            136 non-null    float64
 9   1965            147 non-null    float64
 10  1966            150 non-null    float64
 11  1967            153 non-null    float64
 12  1968            158 non-null    float64
 13  1969            158 non-null    float64
 14  1970            167 non-null    float64
 15  1971            170 non-null    float64
 16  1972            170 non-null    float64
 17  1973            170 non-null    flo

In [687]:
gdp.isnull().sum()

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              134
                 ... 
2019                9
2020                9
2021               15
2022               33
Unnamed: 67       266
Length: 68, dtype: int64

In [688]:
gdp.duplicated().sum()

0

In [689]:
gdp.nunique()

Country Name      266
Country Code      266
Indicator Name      1
Indicator Code      1
1960              131
                 ... 
2019              255
2020              255
2021              249
2022              231
Unnamed: 67         0
Length: 68, dtype: int64

#### Observation
1. The column names should be converted to snake case for consistency.
2. Both the `Indicator Name` and `Indicator Code` columns appear to contain only a single value, suggesting they may be redundant.
3. There are `null` values present in the *years* columns.
4. The dataframe does not contain any duplicate entries.
5. The column names are clear and self-explanatory.


#### Data cleaning and manipulation

In [690]:
def change_str_to_snake_case(string):
    return string.replace(' ', '_').replace('-', '_').lower()

gdp.rename(columns=change_str_to_snake_case, inplace=True)

In [691]:
renamed_countries = {
    'Korea, Rep.': 'South Korea',
    'Iran, Islamic Rep.': 'Iran',
    'Hong Kong SAR, China': 'Hong Kong'
}
gdp.country_name = gdp.country_name.replace(renamed_countries)

#### Reading the Sciamgo Journal and Country Rank data

In [692]:
scim_en = pd.read_excel('./datasets/scimago_journal_country_rank.xlsx')
scim_en.head(10)

,Rank,Country,Region,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,Asiatic Region,360468,358777,3947871,2705774,10.95,308
1,2,United States,Northern America,199442,195042,3068926,881789,15.39,422
2,3,India,Asiatic Region,76103,74167,760964,280893,10.00,217
3,4,Japan,Asiatic Region,56249,55680,633294,136132,11.26,217
4,5,United Kingdom,Western Europe,52572,51156,909276,151672,17.30,267
5,6,Germany,Western Europe,47781,46767,641717,133693,13.43,230
6,7,Russian Federation,Eastern Europe,43567,43290,175721,79765,4.03,103
7,8,Canada,Northern America,39036,38276,787010,125333,20.16,263
8,9,Italy,Western Europe,35991,34424,529459,123042,14.71,192
9,10,South Korea,Asiatic Region,35294,35005,503147,87529,14.26,182


#### Inspecting the data

In [693]:
scim_en.shape

(212, 9)

In [694]:
scim_en.columns

Index(['Rank', 'Country', 'Region', 'Documents', 'Citable documents',
       'Citations', 'Self-citations', 'Citations per document', 'H index'],
      dtype='object')

In [695]:
scim_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Rank                    212 non-null    int64  
 1   Country                 212 non-null    object 
 2   Region                  212 non-null    object 
 3   Documents               212 non-null    int64  
 4   Citable documents       212 non-null    int64  
 5   Citations               212 non-null    int64  
 6   Self-citations          212 non-null    int64  
 7   Citations per document  212 non-null    float64
 8   H index                 212 non-null    int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 15.0+ KB


In [696]:
scim_en.isnull().sum()

Rank                      0
Country                   0
Region                    0
Documents                 0
Citable documents         0
Citations                 0
Self-citations            0
Citations per document    0
H index                   0
dtype: int64

In [697]:
scim_en.duplicated().sum()

0

#### Observation
1. The dataframe comprises 9 columns and 212 rows.
2. To maintain consistency, it's advisable to convert the column names to snake case.
3. No null values are detected within the columns.
4. The dataframe is free of any duplicate entries.
5. The data types of the columns align with the nature of their respective data.

To enhance this dataframe, a few minor adjustments are needed.

#### Data manipulation

In [698]:
scim_en.rename(columns=change_str_to_snake_case, inplace=True)

### Question 1

In [699]:
gdp_years = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
def answer_one():
    return (scim_en.sort_values(by='rank').iloc[:15, :]
            .merge(energy, on='country')
            .merge(gdp[['country_name', *gdp_years]], left_on='country', right_on='country_name')
            .drop(['country_name', 'region'], axis=1)
            .set_index('country'))

answer_one()

,rank,documents,citable_documents,citations,self_citations,citations_per_document,h_index,energy_supply,energy_supply_per_capita,renewable_%,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
country,,,,,,,,,,,,,,,,,,,,
China,1,360468,358777,3947871,2705774,10.95,308,1.271910e+11,93.0,19.754910,2.752119e+12,3.550328e+12,4.594337e+12,5.101691e+12,6.087192e+12,7.551545e+12,8.532186e+12,9.570471e+12,1.047562e+13,1.106157e+13
United States,2,199442,195042,3068926,881789,15.39,422,9.083800e+10,286.0,11.570980,1.381559e+13,1.447423e+13,1.476986e+13,1.447806e+13,1.504896e+13,1.559973e+13,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13
India,3,76103,74167,760964,280893,10.00,217,3.319500e+10,26.0,14.969080,9.402599e+11,1.216736e+12,1.198895e+12,1.341888e+12,1.675616e+12,1.823052e+12,1.827638e+12,1.856721e+12,2.039126e+12,2.103588e+12
Japan,4,56249,55680,633294,136132,11.26,217,1.898400e+10,149.0,10.232820,4.601663e+12,4.579750e+12,5.106679e+12,5.289494e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,52572,51156,909276,151672,17.30,267,7.920000e+09,124.0,10.600470,2.709978e+12,3.092996e+12,2.931684e+12,2.417566e+12,2.491397e+12,2.666403e+12,2.706341e+12,2.786315e+12,3.065223e+12,2.934858e+12
Germany,6,47781,46767,641717,133693,13.43,230,1.326100e+10,165.0,17.901530,2.994704e+12,3.425578e+12,3.745264e+12,3.411261e+12,3.399668e+12,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12
Russian Federation,7,43567,43290,175721,79765,4.03,103,3.070900e+10,214.0,17.288680,9.899321e+11,1.299703e+12,1.660848e+12,1.222646e+12,1.524917e+12,2.045923e+12,2.208294e+12,2.292470e+12,2.059242e+12,1.363482e+12
Canada,8,39036,38276,787010,125333,20.16,263,1.043100e+10,296.0,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,35991,34424,529459,123042,14.71,192,6.530000e+09,109.0,33.667230,1.949552e+12,2.213102e+12,2.408655e+12,2.199929e+12,2.136100e+12,2.294994e+12,2.086958e+12,2.141924e+12,2.162010e+12,1.836638e+12


### Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [700]:
def answer_two():
    top_15 = answer_one()
    avg_gdp = top_15[gdp_years].mean(axis=1)
    return avg_gdp.sort_values(ascending=False)
answer_two()

country
United States         1.570403e+13
China                 6.927707e+12
Japan                 5.239642e+12
Germany               3.523342e+12
United Kingdom        2.780276e+12
France                2.691337e+12
Italy                 2.142986e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.400886e+12
South Korea           1.221372e+12
Australia             1.207513e+12
Iran                  4.563261e+11
dtype: float64

### Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [707]:
def answer_three():
    top_15 = answer_one()
    avg_gdp = answer_two()
    return np.round(top_15.loc[avg_gdp.index[5], gdp_years[-1]] - top_15.loc[avg_gdp.index[5], gdp_years[0]])
answer_three()

118652421858.0

### Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [702]:
def answer_four():
    top_15 = answer_one()
    top_15['citations_ratio'] = top_15.self_citations / top_15.citations
    max_value = top_15.citations_ratio.sort_values(ascending=False).iloc[:1]
    return max_value.index[0], np.round(max_value.iloc[0], 4)
answer_four()

('China', 0.6854)

### Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [703]:
def get_top_15_with_population():
    top_15 = answer_one()
    top_15['population'] = (top_15.energy_supply / top_15.energy_supply_per_capita).round()
    return top_15

def answer_five():
    top_15 = get_top_15_with_population()
    top_15.sort_values(by='population', ascending=False, inplace=True)
    return top_15.iloc[[2]].index[0]
answer_five()

'United States'

### Question 6
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [704]:
def answer_six():
    top_15 = get_top_15_with_population()
    top_15['citable_documents_per_capita'] = top_15.citable_documents / top_15.population
    return np.round(top_15['citable_documents_per_capita'].corr(top_15['energy_supply_per_capita']), 2)
answer_six()

0.71

### Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [705]:
country_to_continent = {
    'China':'Asia',
    'United States':'North America',
    'Japan':'Asia',
    'United Kingdom':'Europe',
    'Russian Federation':'Europe',
    'Canada':'North America',
    'Germany':'Europe',
    'India':'Asia',
    'France':'Europe',
    'South Korea':'Asia',
    'Italy':'Europe',
    'Spain':'Europe',
    'Iran':'Asia',
    'Australia':'Australia',
    'Brazil':'South America'
}
def std(x): return np.std(x)
def answer_seven():
    top_15 = get_top_15_with_population()
    print(top_15.loc['Australia', 'population'].std())
    top_15['continent'] = [country_to_continent[country] for country in top_15.index.to_list()]
    continent_stat = top_15.reset_index().pivot_table(
        index='continent', values=['country', 'population'], 
        aggfunc={'country': 'count', 'population': ['sum', 'mean', std]}
    )
    return continent_stat
answer_seven()

0.0


country    population                            
                count          mean           std           sum
continent                                                      
Asia                5  5.797333e+08  6.074036e+08  2.898666e+09
Australia           1  2.331602e+07  0.000000e+00  2.331602e+07
Europe              6  7.632161e+07  3.162885e+07  4.579297e+08
North America       2  1.764276e+08  1.411878e+08  3.528552e+08
South America       1  2.059153e+08  0.000000e+00  2.059153e+08